In [1]:
import pandas as pd
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.agent import ReActAgent
from IPython.display import Markdown, display
from duckduckgo_search import DDGS

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
llm = OpenAI(model="gpt-4o")
Settings.llm = llm

## Save links

In [11]:
law_links = {
    'regulations': [
        'General Data Protection Regulation (GDPR)',
        'California Consumer Privacy Act (CCPA)',
        'Personal Information Protection and Electronic Documents Act (PIPEDA)'
    ],
    'links': [
        'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32016L0680',
        'https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5',
        'https://laws-lois.justice.gc.ca/ENG/ACTS/P-8.6/FullText.html'
    ]
}

In [12]:
pd.DataFrame(law_links).to_csv('../data/regulations/links.csv',
                               index=False,
                               encoding='utf-8')

## To QueryEngines

In [4]:
links_df = pd.read_csv('../data/regulations/links.csv', encoding='utf-8')
links_df

,regulations,links
0,General Data Protection Regulation (GDPR),https://eur-lex.europa.eu/legal-content/EN/TXT...
1,California Consumer Privacy Act (CCPA),https://leginfo.legislature.ca.gov/faces/codes...
2,Personal Information Protection and Electronic...,https://laws-lois.justice.gc.ca/ENG/ACTS/P-8.6...


In [5]:
regulation_text = SimpleWebPageReader(html_to_text=True).load_data(
    links_df['links'].to_list())

In [6]:
regulation_query_tools = [
    QueryEngineTool(
        query_engine=VectorStoreIndex.from_documents(
            [regulation_text[i]]).as_query_engine(llm=llm),
        metadata=ToolMetadata(
            name=links_df.loc[i, 'regulations'],
            description=("refer the text when you want to make something be "
                         f"{links_df.loc[i, 'regulations']} compliant"),
        ),
    ) for i in range(len(regulation_text))
]

In [7]:
regulation_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=regulation_query_tools, llm=llm, verbose=True)

In [9]:
output = regulation_query_engine.query(
    """I want to write a privacy policy, which should be GDPR and CCPA compliant.
    
    It includes the following sections:
    1. Introduction
    2. Identity and Contact Details of the Controller
    3. Data Collection and Use
    4. Data Subject Rights
    5. Data Sharing and Transfers
    6. Data Retention
    7. Security Measures
    8. Automated Decision-Making and Profiling
    9. Cookies and Tracking Technologies
    10. Changes to the Privacy Policy
    11. Contact Information

    Now given the sections, I want to know what should be included in each section to be compliant with GDPR and CCPA.
    List them as bullet points.
    """)
display(Markdown(f"<b>{output.response}</b>"))

Generated 22 sub questions.
[General Data Protection Regulation (GDPR)] Q: What should be included in the Introduction section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Identity and Contact Details of the Controller section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Data Collection and Use section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Data Subject Rights section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Data Sharing and Transfers section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Data Retention section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Security Measures section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What 

<b>To ensure your privacy policy is compliant with both GDPR and CCPA, each section should include the following elements:

### 1. Introduction
- **GDPR**:
  - Identity of the data controller.
  - Existence of the data processing operation.
  - Purposes of the data processing.
  - Right of the data subject to lodge a complaint.
  - Right of the data subject to request access to, rectification or erasure of personal data, or restriction of processing.
- **CCPA**:
  - Categories of personal information collected.
  - Purposes for collection.
  - Disclosure of sale or sharing of personal information.
  - Categories and purposes of sensitive personal information collected.
  - Retention period for each category of personal information.
  - Opt-out information.
  - Accessibility for consumers with disabilities and in the primary language used to interact with the consumer.

### 2. Identity and Contact Details of the Controller
- **GDPR**:
  - Identity of the controller.
  - Existence of the processing operation.
  - Purposes of the processing.
  - Right to lodge a complaint.
  - Right to request access to and rectification or erasure of personal data or restriction of processing.
  - Legal basis for the processing.
  - Data storage duration.
- **CCPA**:
  - Name or names of the owner, director, officer, or management employee.
  - Contact information, including business title.

### 3. Data Collection and Use
- **GDPR**:
  - Identity of the controller.
  - Existence of the processing operation.
  - Purposes of the processing.
  - Legal basis for processing.
  - Categories of personal data.
  - Data subject rights.
  - Safeguards for sensitive data.
  - Time limits for data storage.
  - Quality and accuracy of data.
  - Automated decision-making information.
  - Right to lodge a complaint.
  - Additional information for identity verification.
- **CCPA**:
  - Categories of personal information collected.
  - Categories of sensitive personal information collected.
  - Retention period for each category of personal information.
  - Purpose of collection and use.
  - Third-party agreements.
  - Security measures.
  - Consumer rights to request deletion.
  - Opt-out options.

### 4. Data Subject Rights
- **GDPR**:
  - Identity of the controller.
  - Existence of the processing operation.
  - Purposes of the processing.
  - Right to lodge a complaint.
  - Right to request access to and rectification or erasure of personal data or restriction of processing.
  - Legal basis for the processing.
  - Data storage duration.
  - How to exercise these rights.
- **CCPA**:
  - Right to request correction of inaccurate personal information.
  - Right to know what personal information is being collected.
  - Right to opt-out of the sale or sharing of personal information.
  - Right to limit the use or disclosure of sensitive personal information.
  - How to exercise these rights.
  - Disclosure of financial incentives.

### 5. Data Sharing and Transfers
- **GDPR**:
  - Conditions for transfers to third countries or international organizations.
  - Protections for sensitive personal data.
  - Automated decision-making safeguards.
  - Information accessibility.
  - Mechanisms for data subject rights.
  - Identity confirmation procedures.
  - Mandatory information.
  - Adequacy decisions.
  - Periodic review and monitoring.
  - Consultation and remediation procedures.
  - Appropriate safeguards for non-adequacy decision transfers.
- **CCPA**:
  - Identification of the consumer and association of information.
  - Categories of personal information sold or shared.
  - Categories of personal information disclosed for a business purpose.
  - Description of consumer rights and methods for submitting requests.
  - Lists of categories of personal information collected, sold, shared, and disclosed.
  - Opt-out information.
  - Terms of financial incentives.

### 6. Data Retention
- **GDPR**:
  - Purpose limitation.
  - Time limits for data retention.
  - Review procedures.
  - Legal basis for processing and storage duration.
  - Data subject rights.
  - Compliance responsibility.
- **CCPA**:
  - Retention period for each category of personal information.
  - Criteria used to determine retention period.
  - Statement on retention duration relative to the disclosed purpose.

### 7. Security Measures
- **GDPR**:
  - Evaluation of risks inherent in data processing.
  - Implementation of measures to mitigate risks.
  - Appropriate level of security, including confidentiality.
  - Technical and organizational measures (e.g., encryption).
- **CCPA**:
  - Reasonable security procedures and practices.
  - Protection against unauthorized or illegal access, destruction, use, modification, or disclosure.

### 8. Automated Decision-Making and Profiling
- **GDPR**:
  - Transparency and information about automated decision-making.
  - Legal basis for automated decision-making.
  - Rights of data subjects (human intervention, express point of view, contest decision).
  - Safeguards for fairness, transparency, and non-discrimination.
  - Data minimization and accuracy.
  - Data Protection Impact Assessments (DPIAs).
  - Security measures.
  - Regular reviews.
- **CCPA**:
  - Opt-out links for sale or sharing of personal information.
  - Description of consumer rights.
  - Mechanism for exercising rights without creating an account.
  - Use of personal information solely for compliance with requests.

### 9. Cookies and Tracking Technologies
- **GDPR**:
  - Types of cookies and tracking technologies used.
  - Purpose of each cookie and tracking technology.
  - Data collected.
  - Legal basis for processing.
  - Duration of data storage.
  - Third-party access.
  - User rights.
  - Instructions for managing cookies.
  - Impact of disabling cookies.
  - Contact information.
- **CCPA**:
  - Description of consumer rights regarding cookies and tracking technologies.
  - Opt-out links for sale or sharing of personal information.
  - Information on financial incentives related to personal information.

### 10. Changes to the Privacy Policy
- **GDPR**:
  - Notification of changes.
  - Effective date of changes.
  - Description of changes.
  - User rights regarding changes.
  - Access to previous versions.
  - Contact information.
- **CCPA**:
  - Disclosure of consumer rights.
  - Procedures for requests.
  - Business obligations.
  - Frequency and circumstances for requests.
  - Opt-out information.
  - Updates and notifications.
  - Accessibility.

### 11. Contact Information
- **GDPR**:
  - Identity of the controller.
  - Existence of the processing operation.
  - Purposes of the processing.
  - Right to lodge a complaint.
  - Right to request access to and rectification or erasure of personal data or restriction of processing.
  - Legal basis for processing.
  - Data storage duration.
- **CCPA**:
  - Contact information for handling consumer inquiries.
  - Opt-out links for sale or sharing of personal information.
  - Links to limit the use or disclosure of sensitive personal information.</b>